<a href="https://colab.research.google.com/github/UiinKim/Crawling/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EC%8A%A4%EB%A7%88%ED%8A%B8%EC%8A%A4%ED%86%A0%EC%96%B4%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
from time import sleep
import re, requests, csv
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
keys=Keys()

url='https://brand.naver.com/roundlab/products/5956405279'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
chrome_driver_path = '/Users/UiinKim/.ipython/chromedriver'
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(3)

page=requests.get(url)
while True:
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

ModuleNotFoundError: ignored

In [ ]:
html=driver.page_source
soup=bs(html, 'html.parser')
text=[]
result=[]
for tag1 in soup.find_all(class_='_1iuv6pLHMD'):
    text.append(tag1.text)
for tag2 in soup.find_all(class_='ABROiEshTD'):
    result.append(tag2.text)
for tag, name in zip(text, result):
    if tag=='모델명':
        model=name
    if tag=='브랜드':
        brand=name
    if tag=='상품번호':
        number=name
#model='클래식 에센스 인크리멘트 50 mL'
#brand='아벤느'
print(model, brand, number)

In [ ]:
count=0
stop=int(input('전체 리뷰 수를 입력해주세요 : '))
next_btn = ['a:nth-child(2)', 'a:nth-child(3)', 'a:nth-child(4)', 'a:nth-child(5)', 'a:nth-child(6)', 'a:nth-child(7)', 'a:nth-child(8)', 'a:nth-child(9)', 'a:nth-child(10)', 'a:nth-child(11)','a.fAUKm1ewwo._2Ar8-aEUTq']
review_list=[]
while count<stop:
    for pagenum in next_btn:
        driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > '+str(pagenum)+'').send_keys(Keys.ENTER)
        time.sleep(5)
        for i in range(0,20):
            html=driver.page_source
            soup=bs(html, 'html.parser')
            review=soup.find_all('div', class_='_19SE1Dnqkf')
            review=review[i].text
            review=re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣\s]', " ", review)
            review=re.sub(r'한달사용기', '',review)
            review=re.sub(r'BEST', '',review)
            review=review.replace('\n', ' ')
            review=re.sub(r' {2,}', ' ', review)
            review_list.append(review)
            count+=1
            if(count==stop):
                break
        if count==stop:
            break


review_list

In [ ]:
df=pd.DataFrame({'brand_en':brand, 'product_name':model, 'product_num': number, 'review':review_list})
df.to_csv('c:/Users/UiinKim/crawling_2023/화장품추천시스템/라운드랩/라운드랩_%s.csv'%(model), encoding='cp949')
print('저장 완료되었습니다.')